In [ ]:
# kaggle data: https://www.kaggle.com/datasets/jbencina/department-of-justice-20092018-press-releases

In [ ]:
import pandas as pd
import numpy as np
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM,Dense,Dropout,Embedding
import tensorflow as tf
from tensorflow.keras import Sequential
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
add='/content/drive/My Drive/Colab Notebooks/Department of Justice 2009-2018 Press Releases.json'
df = pd.read_json(add, lines=True)
df.head()

,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],[National Security Division (NSD)]
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],[Environment and Natural Resources Division]
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],[Environment and Natural Resources Division]
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],[Environment and Natural Resources Division]
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],[Environment and Natural Resources Division]


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13087 entries, 0 to 13086
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12810 non-null  object
 1   title       13087 non-null  object
 2   contents    13087 non-null  object
 3   date        13087 non-null  object
 4   topics      13087 non-null  object
 5   components  13087 non-null  object
dtypes: object(6)
memory usage: 613.6+ KB


In [ ]:
df['components']=df['components'].astype('string')
df.head()

,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],['National Security Division (NSD)']
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],['Environment and Natural Resources Division']
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],['Environment and Natural Resources Division']
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],['Environment and Natural Resources Division']
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],['Environment and Natural Resources Division']


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13087 entries, 0 to 13086
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          12810 non-null  object
 1   title       13087 non-null  object
 2   contents    13087 non-null  object
 3   date        13087 non-null  object
 4   topics      13087 non-null  object
 5   components  13087 non-null  string
dtypes: object(5), string(1)
memory usage: 613.6+ KB


In [ ]:
df['components'].value_counts()

['Criminal Division']                                                                                                                    2680
['Tax Division']                                                                                                                         1862
['Civil Division']                                                                                                                        926
['Civil Rights Division']                                                                                                                 862
['Office of the Attorney General']                                                                                                        822
                                                                                                                                         ... 
['Civil Rights Division', 'Civil Rights - Special Litigation Section', 'Office of the Attorney General', 'USAO - Illinois, Northern']       1
['Civi

In [ ]:
df['components'].value_counts()[:10]

['Criminal Division']                                                    2680
['Tax Division']                                                         1862
['Civil Division']                                                        926
['Civil Rights Division']                                                 862
['Office of the Attorney General']                                        822
['Environment and Natural Resources Division']                            798
['Antitrust Division']                                                    798
['Civil Rights Division', 'Civil Rights - Criminal Section']              554
['National Security Division (NSD)']                                      330
['Civil Rights Division', 'Civil Rights - Disability Rights Section']     184
Name: components, dtype: Int64

In [ ]:
x=["['Criminal Division']","['Tax Division']","['Civil Division']","['Civil Division']","['Civil Rights Division']","['Antitrust Division']","['Environment and Natural Resources Division']","['Civil Rights Division, Civil Rights - Criminal Section']","['National Security Division (NSD)']","['Civil Rights Division, Civil Rights - Disability Rights Section']"]
index_names = df[(df['components']!=x[0]) & (df['components']!=x[1]) & (df['components']!=x[2]) & (df['components']!=x[3]) & (df['components']!=x[4]) &
                  (df['components']!=x[5]) & (df['components']!=x[6]) & (df['components']!=x[7]) & (df['components']!=x[8]) & (df['components']!=x[9]) ].index
print(len(index_names))
df.drop(index_names, inplace = True)
df.head()

4831


,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],['National Security Division (NSD)']
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],['Environment and Natural Resources Division']
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],['Environment and Natural Resources Division']
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],['Environment and Natural Resources Division']
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],['Environment and Natural Resources Division']


In [ ]:
requires = {"['Criminal Division']":1,"['Tax Division']":2,"['Civil Division']":3,"['Civil Division']":4,"['Civil Rights Division']":5,"['Antitrust Division']":6,"['Environment and Natural Resources Division']":7,"['Civil Rights Division, Civil Rights - Criminal Section']":8,"['National Security Division (NSD)']":9,"['Civil Rights Division, Civil Rights - Disability Rights Section']":10}
df['components']=df['components'].replace(requires)
df.head()

,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"PORTLAND, Oregon. – Mohamed Osman Mohamud, 23,...",2014-10-01T00:00:00-04:00,[],9
1,12-919,$1 Million in Restitution Payments Announced t...,WASHINGTON – North Carolina’s Waccamaw River...,2012-07-25T00:00:00-04:00,[],7
2,11-1002,$1 Million Settlement Reached for Natural Reso...,BOSTON– A $1-million settlement has been...,2011-08-03T00:00:00-04:00,[],7
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,WASHINGTON—A federal grand jury in Las Vegas...,2010-01-08T00:00:00-05:00,[],7
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"The U.S. Department of Justice, the U.S. Envir...",2018-07-09T00:00:00-04:00,[Environment],7


In [ ]:
def toLower(sentence):
    return sentence.lower()

def tokenizer(sentence):
    tokens = list(set(nltk.word_tokenize(sentence)))
    return tokens

def stopwords_removal(tokens):
    stop_words = nltk.corpus.stopwords.words('english')
    stop_words.extend([',','?','""',"''",'.','!', "'",'"',"'d","'ll",'[',']','--',':',';','///','@', '``',
                       '#', '$', '%', '&', "'re", "'s", '(', ')', '*', '**', '**the', '-', '/', '//',
                       '§', '§§','...','–', '—', '‘', '’', '“', '”', '•', '─',"'m", "'ve", '***'])
    filtered_tokens = [i for i in tokens if not i in stop_words]
    return filtered_tokens

def stemming(tokens):
    stemmer = nltk.stem.porter.PorterStemmer()
    stemmed_tokens = [stemmer.stem(i) for i in tokens]
    return stemmed_tokens

def pre_process(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    tokens = stopwords_removal(tokens)
    stems = stemming(tokens)
    return stems

In [ ]:
df['contents']=df['contents'].apply(pre_process)
df['contents']

0        [portland, oregon, moham, osman, mohamud, 23, ...
1        [washington, north, carolina, waccamaw, river,...
2        [boston–, A, 1-million, settlement, reach, nat...
3        [washington—a, feder, grand, juri, la, vega, t...
4        [the, u.s., depart, justic, u.s., environment,...
                               ...                        
13074    [the, depart, justic, u.s., environment, prote...
13075    [yamada, manufactur, co., ltd., agre, plead, g...
13076    [washington, two, japanes, supplier, automot, ...
13077    [washington, An, execut, tokyo-bas, yazaki, co...
13082    [the, depart, justic, u.s., environment, prote...
Name: contents, Length: 8256, dtype: object

In [ ]:
df.reset_index(inplace = True)
del ( df['index'])

In [ ]:
df

,id,title,contents,date,topics,components
0,None,Convicted Bomb Plotter Sentenced to 30 Years,"[portland, oregon, moham, osman, mohamud, 23, ...",2014-10-01T00:00:00-04:00,[],9
1,12-919,$1 Million in Restitution Payments Announced t...,"[washington, north, carolina, waccamaw, river,...",2012-07-25T00:00:00-04:00,[],7
2,11-1002,$1 Million Settlement Reached for Natural Reso...,"[boston–, A, 1-million, settlement, reach, nat...",2011-08-03T00:00:00-04:00,[],7
3,10-015,10 Las Vegas Men Indicted \r\nfor Falsifying V...,"[washington—a, feder, grand, juri, la, vega, t...",2010-01-08T00:00:00-05:00,[],7
4,18-898,$100 Million Settlement Will Speed Cleanup Wor...,"[the, u.s., depart, justic, u.s., environment,...",2018-07-09T00:00:00-04:00,[Environment],7
...,...,...,...,...,...,...
8251,14-1454,XTO Energy Inc. to Restore Areas Damaged by Na...,"[the, depart, justic, u.s., environment, prote...",2014-12-22T00:00:00-05:00,[],7
8252,15-525,Yamada Manufacturing Co. Agrees to Plead Guilt...,"[yamada, manufactur, co., ltd., agre, plead, g...",2015-04-28T00:00:00-04:00,[Antitrust],6
8253,12-128,"Yazaki Corp., Denso Corp. and Four Yazaki Exec...","[washington, two, japanes, supplier, automot, ...",2012-01-30T00:00:00-05:00,[],6
8254,12-1013,Yazaki Executive Agrees to Plead Guilty to Pri...,"[washington, An, execut, tokyo-bas, yazaki, co...",2012-08-16T00:00:00-04:00,[],6


In [ ]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',lower=True, split=' ', char_level=False, oov_token='<oov>',)
tokenizer.fit_on_texts(list(df['contents']))

In [ ]:
sequences = tokenizer.texts_to_sequences(list(df['contents']))
max=np.max([len(sequence) for sequence in sequences])
print("Longest sentence length :- ",max)
word_index = tokenizer.word_index
print('Number of  unique tokens :- %s' % len(word_index))

Longest sentence length :-  2506
Number of  unique tokens :- 48507


In [ ]:
data = tf.keras.preprocessing.sequence.pad_sequences(sequences, maxlen=max, dtype='int32', padding='pre',truncating='pre', value=0.0)
print(data.shape)

(8256, 2506)


In [ ]:
labels = tf.keras.utils.to_categorical(df['components'])

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(data, labels, test_size=0.25, random_state=14)

In [ ]:
model=Sequential()
model.add(Embedding(len(word_index)+1,64,input_length=max))
model.add(LSTM(256,return_sequences=False))
model.add(Dropout(0.25))
model.add(Dense(1024,activation='relu'))
model.add(Dense(10,activation='softmax'))

In [ ]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [ ]:
model.fit(X_train,Y_train,batch_size=128,epochs=35,validation_data=(X_test,Y_test))

Epoch 1/35
49/49 [==============================] - 31s 575ms/step - loss: 1.9291 - accuracy: 0.3717 - val_loss: 0.9324 - val_accuracy: 0.6526
Epoch 2/35
49/49 [==============================] - 27s 557ms/step - loss: 0.6315 - accuracy: 0.7737 - val_loss: 0.2639 - val_accuracy: 0.9230
Epoch 3/35
49/49 [==============================] - 27s 562ms/step - loss: 0.1587 - accuracy: 0.9540 - val_loss: 0.1937 - val_accuracy: 0.9530
Epoch 4/35
49/49 [==============================] - 28s 563ms/step - loss: 0.0874 - accuracy: 0.9811 - val_loss: 0.1651 - val_accuracy: 0.9632
Epoch 5/35
49/49 [==============================] - 28s 564ms/step - loss: 0.0798 - accuracy: 0.9851 - val_loss: 0.2466 - val_accuracy: 0.9312
Epoch 6/35
49/49 [==============================] - 27s 560ms/step - loss: 0.1130 - accuracy: 0.9709 - val_loss: 0.2270 - val_accuracy: 0.9583
Epoch 7/35
49/49 [==============================] - 27s 561ms/step - loss: 0.0413 - accuracy: 0.9889 - val_loss: 0.1976 - val_accuracy: 0.9564